In [1]:
import pandas as pd
import pymysql
from decimal import Decimal, ROUND_HALF_UP

In [2]:
# Подключение к БД
connection = pymysql.connect(host='84.201.138.51',
                             port=3306,
                             user='sl_dev_vacancy',
                             password='q5Hcsp@H',
                             database='tv_dashboards',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

# Запрос и загрузка данных
query = '''
    SELECT New_brand, Year, Month, Week, 
           REPLACE(TVCompany, '(СЕТЕВОЕ ВЕЩАНИЕ)', '') AS TVCompany,
           Programme_genre,
           SUM(Quantity) AS Quantity,
           SUM(Consolidated_cost) AS Consolidated_cost,
           SUM(`TVR_All_18+`) AS TVR_All_18_plus
    FROM test_base_tv
    GROUP BY New_brand, Year, Month, Week, TVCompany, Programme_genre
'''
# REPLACE(TVCompany, '(СЕТЕВОЕ ВЕЩАНИЕ)', '') AS TVCompany,- Убирает словочочетание "СЕТЕВОЕ ВЕЩАНИЕ" по полю "TVCompany"

with connection.cursor() as cursor:
    cursor.execute(query)
    result = cursor.fetchall()

df = pd.DataFrame(result)



In [3]:
df

,New_brand,Year,Month,Week,TVCompany,Programme_genre,Quantity,Consolidated_cost,TVR_All_18_plus
0,АФОБАЗОЛ,2017,1,1,МУЗ ТВ,Музыкальная программа,3,15521.00,0.3210
1,АФОБАЗОЛ,2017,1,2,ANIMAL PLANET,Прочее,22,78804.00,0.7337
2,АФОБАЗОЛ,2017,1,2,БОБЁР,Прочее,24,0.00,0.1548
3,АФОБАЗОЛ,2017,1,2,ДОМАШНИЙ,Кинопрограмма,16,725905.00,10.0166
4,АФОБАЗОЛ,2017,1,2,ДОМАШНИЙ,Развлекательная программа,2,45076.00,0.5084
...,...,...,...,...,...,...,...,...,...
60793,ЭВАЛАР ANTI-AGE МАГНИЙ ХЕЛАТ,2023,3,13,ТВ-3,Кинофильм,5,481991.00,2.1279
60794,ЭВАЛАР ANTI-AGE МАГНИЙ ХЕЛАТ,2023,3,13,ТВ-3,Познавательная программа,4,427357.00,2.4019
60795,ЭВАЛАР ANTI-AGE МАГНИЙ ХЕЛАТ,2023,3,13,ТВ-3,Социально-политическая программа,1,96719.00,0.6290
60796,ЭВАЛАР ANTI-AGE МАГНИЙ ХЕЛАТ,2023,3,13,ТВ-3,Телесериал,13,1695651.00,8.8612


In [59]:
#Добавление столбца НДС
def calculate_vat(row):
    vat_rate = Decimal('0.18') if row['Year'] < 2019 else Decimal('0.20')
    return (row['Consolidated_cost'] * vat_rate).quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)

df['VAT'] = df.apply(calculate_vat, axis=1)

In [60]:
df

,New_brand,Year,Month,Week,TVCompany,Programme_genre,Quantity,Consolidated_cost,TVR_All_18_plus,VAT
0,АФОБАЗОЛ,2017,1,1,МУЗ ТВ,Музыкальная программа,3,15521.00,0.3210,2793.78
1,АФОБАЗОЛ,2017,1,2,ANIMAL PLANET,Прочее,22,78804.00,0.7337,14184.72
2,АФОБАЗОЛ,2017,1,2,БОБЁР,Прочее,24,0.00,0.1548,0.00
3,АФОБАЗОЛ,2017,1,2,ДОМАШНИЙ,Кинопрограмма,16,725905.00,10.0166,130662.90
4,АФОБАЗОЛ,2017,1,2,ДОМАШНИЙ,Развлекательная программа,2,45076.00,0.5084,8113.68
...,...,...,...,...,...,...,...,...,...,...
60793,ЭВАЛАР ANTI-AGE МАГНИЙ ХЕЛАТ,2023,3,13,ТВ-3,Кинофильм,5,481991.00,2.1279,96398.20
60794,ЭВАЛАР ANTI-AGE МАГНИЙ ХЕЛАТ,2023,3,13,ТВ-3,Познавательная программа,4,427357.00,2.4019,85471.40
60795,ЭВАЛАР ANTI-AGE МАГНИЙ ХЕЛАТ,2023,3,13,ТВ-3,Социально-политическая программа,1,96719.00,0.6290,19343.80
60796,ЭВАЛАР ANTI-AGE МАГНИЙ ХЕЛАТ,2023,3,13,ТВ-3,Телесериал,13,1695651.00,8.8612,339130.20


In [61]:
# Рассчет среднего недельного TVR_All_18+ по столбцам New_brand/Year
df_brand_year = df.groupby(['New_brand', 'Year'])['TVR_All_18_plus'].mean().reset_index()

In [62]:
df_brand_year

,New_brand,Year,TVR_All_18_plus
0,АФОБАЗОЛ,2017,3.79981
1,АФОБАЗОЛ,2018,4.300268
2,АФОБАЗОЛ,2019,3.313031
3,АФОБАЗОЛ,2020,3.481849
4,АФОБАЗОЛ,2021,3.340046
5,АФОБАЗОЛ,2022,3.457885
6,АФОБАЗОЛ,2023,3.501058
7,ВЕЛСОН,2019,4.566736
8,ВЕЛСОН,2020,4.645007
9,ВЕЛСОН,2021,5.543362


In [63]:
# Рассчет суммарного Consolidated_cost по столбцам Year/TVCompany
df_year_TVcompany = df.groupby(['Year', 'TVCompany'])['Consolidated_cost'].sum().reset_index()

In [64]:
df_year_TVcompany

,Year,TVCompany,Consolidated_cost
0,2017,24_DOC,0.00
1,2017,2X2,7624684.48
2,2017,ANIMAL PLANET,4886663.00
3,2017,BRIDGE,2139.00
4,2017,BRIDGE РУССКИЙ ХИТ,76226.00
...,...,...,...
429,2023,ТВ-3 (ИНТЕРНЕТ),0.00
430,2023,ТНТ,177914475.00
431,2023,ТНТ (ИНТЕРНЕТ),0.00
432,2023,ФЕНИКС+ КИНО,561.00


In [65]:
# Рассчет среднего НДС по столбцу Programme_genre
df_cost_genre = df.groupby('Programme_genre')['VAT'].mean().reset_index()
df_cost_genre = df_cost_genre.sort_values('VAT', ascending=False)

In [66]:
df_cost_genre

,Programme_genre,VAT
3,Кинопрограмма,142453.845352
9,Развлекательная программа,77487.214865
12,Телесериал,75556.924864
8,Прочее,68997.167368
4,Кинофильм,44856.940227
10,Социально-политическая программа,41929.936274
0,Анимация,38297.674219
5,Музыкальная программа,37071.682914
6,Новости,27708.171531
2,Документальная программа,22253.15896


In [67]:
#Запись в Excel
with pd.ExcelWriter('tv_dashboards_results.xlsx') as writer:
    df.to_excel(writer, sheet_name='Main Table', index=False)
    df_brand_year.to_excel(writer, sheet_name='Brand-Year', index=False)
    df_year_TVcompany.to_excel(writer, sheet_name='Year-Company', index=False)
    df_cost_genre.to_excel(writer, sheet_name='Genre-Cost', index=False)